# Мост (bridge)

Мост — это структурный паттерн проектирования, который разделяет один или несколько классов на две отдельные иерархии — абстракцию и реализацию, позволяя изменять их независимо друг от друга.


## Пример

Автоматизируем продажу смартфонов. Итоговая цена смартфона зависит от страны производителя (Индия, Китай), сезонности (допустим, летом продаются лучше). Для демонстрации, будут использованы 2 модели смартфонов от DNS и Samsung.


### Проблема

**Наивная реализация.** Генерируем отдельный класс, для каждого сочетания: модель смартфона, производитель, сезонность. Получили 8 классов (2 _ 2 _ 2). При добавлении всего одного производителя, получим 12 классов (2 _ 3 _ 2).


In [7]:
import abc


class AbsSmartphone(abc.ABC):
    @abc.abstractmethod
    def calc_price(self) -> float:
        """
        В дальнейших реализациях я захардкодил конкретные значения,
        но в боевых проектах, там может находиться сложная логика
        """
        ...


class IndiaSummerDNS(AbsSmartphone):
    def calc_price(self) -> float:
        return 1.0 * 0.9 * 7_000


class IndiaSummerSamsung(AbsSmartphone):
    def calc_price(self) -> float:
        return 1.0 * 0.9 * 13_000


class IndiaWinterDNS(AbsSmartphone):
    def calc_price(self) -> float:
        return 1.0 * 1.0 * 7_000


class IndiaWinterSamsung(AbsSmartphone):
    def calc_price(self) -> float:
        return 1.0 * 1.0 * 13_000


class ChinaSummerDNS(AbsSmartphone):
    def calc_price(self) -> float:
        return 0.8 * 0.9 * 7_000


class ChinaSummerSamsung(AbsSmartphone):
    def calc_price(self) -> float:
        return 0.8 * 0.9 * 13_000


class ChinaWinterDNS(AbsSmartphone):
    def calc_price(self) -> float:
        return 0.8 * 1.0 * 7_000


class ChinaWinterSamsung(AbsSmartphone):
    def calc_price(self) -> float:
        return 0.8 * 1.0 * 13_000

In [20]:
smartphone = IndiaSummerSamsung()

print("Цена при покупке SAMSUNG из Индии летом:", smartphone.calc_price())

Цена при покупке SAMSUNG из Индии летом: 11700.0


### Решение

Поборемся с ростом подклассов с помощью выделения абстракций. Т. е. вместо наследования, будем использовать композицию с передачей объектов через инициализатор.


In [16]:
import abc


class AbsCoefficient(abc.ABC):
    @abc.abstractproperty
    def coefficient(self) -> float:
        return 0.9


class AbsManufactureCoefficient(AbsCoefficient):
    """
    Абстракция страны производителя
    """

    ...


class ChinaCoefficient(AbsManufactureCoefficient):
    @property
    def coefficient(self) -> float:
        return 0.8


class IndiaCoefficient(AbsManufactureCoefficient):
    @property
    def coefficient(self) -> float:
        return 1.0


class AbsSeasonalityCoefficient(AbsCoefficient):
    """
    Абстракция сезонности (сезон высоких/низких продаж)
    """

    ...


class SummerCoefficient(AbsSeasonalityCoefficient):
    @property
    def coefficient(self) -> float:
        return 0.9


class WinterCoefficient(AbsSeasonalityCoefficient):
    @property
    def coefficient(self) -> float:
        return 1.0

In [17]:
class AbsSmartphone(abc.ABC):
    @abc.abstractmethod
    def calc_price(self) -> float:
        ...

    def __init__(
        self,
        base_price: float,
        market: AbsManufactureCoefficient,
        seasonality: AbsSeasonalityCoefficient,
    ) -> None:
        """
        Вместо наследования, используем композицию.
        """
        self.base_price = base_price
        self.market = market
        self.seasonality = seasonality


class SmartphoneDNS(AbsSmartphone):
    def calc_price(self) -> float:
        """
        Как я упоминал ранее, здесь может находиться сложная логика, поэтому реализовано
        для каждого класса индивидуально.
        """
        return self.base_price * self.market.coefficient * self.seasonality.coefficient


class SmartphoneSamsung(AbsSmartphone):
    def calc_price(self) -> float:
        return self.base_price * self.market.coefficient * self.seasonality.coefficient

In [19]:
smartphone = SmartphoneSamsung(
    13000,
    IndiaCoefficient(),
    SummerCoefficient(),
)

print("Цена при покупке SAMSUNG из Индии летом:", smartphone.calc_price())

Цена при покупке SAMSUNG из Индии летом: 11700.0
